# Extractive Text Summarization

In this notebook, I implement a simple algorithm to perform extractive text summarization on a given set of documents/sentences. 

The vector representation of each document/sentence/query is the TF-IDF vectorisation. I could have used it out-of-the-box from sklearn but I tried to implement it from first principles.. Just for fun!

In [77]:
# Imports
import math 
import numpy as np
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

    
# english stop words from nltk and wordnet lemmatizer for the word root forms 
stopWords = set(stopwords.words('english'))
stemmer = WordNetLemmatizer()

In [55]:
# Reading the original text - since I have super low computational power, I will take in a small document 
# and perform sentence segmentation and treat each sentence as a document 
with open('text.txt', "r") as f:
    text = f.read()


In [57]:
# Sentences segmentation - could have also used Spacy's 'en' language model 
docs = sent_tokenize(text)
total_docs = len(docs)
print("Found {} docs in the corpus".format(total_docs))

Found 52 docs in the corpus


## Generate Frequency matrix 

The frequency matrix is matrix with documents on the row headers and unique words on the column headers. The matrix is filled with frequency of each word in that document and it's total count in the same document. 

This matrix sevres as the basis for calculating term frequency and inverse document frequency. 

In [58]:
# creating some helper functions

def create_frequency_matrix(docs):
    
    # placeholder for frequency matrix
    term_frequency_matrix = {}
    
    # iterating over the docs 
    for doc in docs: 
        freq_table = {}
        words = word_tokenize(doc)
        for word in words:
            word = word.lower()
            word = stemmer.lemmatize(word)
            if word not in stopWords:
                if word in term_frequency_matrix:
                    freq_table[word]+=1
                else:
                    freq_table[word]=1
        term_frequency_matrix[doc[:15]] = freq_table
                    
    return term_frequency_matrix

In [59]:
# calling the above function of docs --> set of all documents in corpus 

perdoc_term_frequency_matrix = create_term_frequency_matrix(docs)
print(perdoc_term_frequency_matrix)

{'Those Who Are R': {'resili': 1, 'stay': 1, 'game': 1, 'longer': 1, '“': 1, 'mountain': 1, 'truth': 1, 'never': 1, 'climb': 1, 'vain': 1, ':': 1, 'either': 1, 'reach': 1, 'point': 1, 'higher': 1, 'today': 1, ',': 1, 'train': 1, 'power': 1, 'abl': 1, 'tomorrow.': 1, '”': 1, '—': 1, 'friedrich': 1, 'nietzsch': 1, 'challeng': 1, 'setback': 1, 'meant': 1, 'defeat': 1, 'promot': 1, '.': 1}, 'However, I real': {'howev': 1, ',': 1, 'realis': 1, 'mani': 1, 'year': 1, 'defeat': 1, 'crush': 1, 'spirit': 1, 'easier': 1, 'give': 1, 'risk': 1, 'setback': 1, 'disappoint': 1, '.': 1}, 'Have you experi': {'experienc': 1, 'thi': 1, 'befor': 1, '?': 1}, 'To be honest, I': {'honest': 1, ',': 1, '’': 1, 'answer': 1, '.': 1}, 'I can’t tell yo': {'’': 1, 'tell': 1, 'right': 1, 'cours': 1, 'action': 1, ';': 1, 'onli': 1, 'know': 1, '.': 1}, 'However, it’s i': {'howev': 1, ',': 1, '’': 1, 'import': 1, 'discourag': 1, 'failur': 1, 'pursu': 1, 'goal': 1, 'dream': 1, 'sinc': 1, 'mean': 1, 'differ': 1, 'thing': 

## Generate Term Frequency matrix 

Term Frequency is the count of a word in a document divided by the total unique words in that document. Term Frequency makes sense only for a given document. 


In [60]:
# calculate the term frequency 

def term_frequency(doc_term_freq_matrix):
    tf = {}
    for doc, table in doc_term_freq_matrix.items():
        tf[doc] = {}
        total_words = len(table)
        for word, count in table.items():
            term_freq = count/total_words
            tf[doc][word] = term_freq
    return tf
term_frequency = term_frequency(doc_term_frequency_matrix)

## Generate Word Document Frequency matrix

Document Frequency is the count of documents in which a particular word appears. Doc frequency makes sense only in the context of a word. 

In [61]:
# create a table for count of docs per word

def create_doc_frequency_matrix(frequency_matrix):
    df = {}
    
    for doc, table in frequency_matrix.items():
        for word, count in table.items():
            if word in df:
                df[word]+=1
            else:
                df[word]=1
    return df

doc_frequency = create_doc_frequency_matrix(perdoc_term_frequency_matrix)
print(doc_frequency)
    

{'resili': 2, 'stay': 2, 'game': 3, 'longer': 2, '“': 5, 'mountain': 1, 'truth': 1, 'never': 2, 'climb': 1, 'vain': 1, ':': 8, 'either': 1, 'reach': 1, 'point': 2, 'higher': 1, 'today': 1, ',': 22, 'train': 1, 'power': 4, 'abl': 1, 'tomorrow.': 1, '”': 5, '—': 3, 'friedrich': 1, 'nietzsch': 1, 'challeng': 2, 'setback': 2, 'meant': 1, 'defeat': 3, 'promot': 1, '.': 45, 'howev': 2, 'realis': 2, 'mani': 3, 'year': 4, 'crush': 1, 'spirit': 1, 'easier': 1, 'give': 4, 'risk': 1, 'disappoint': 2, 'experienc': 1, 'thi': 4, 'befor': 2, '?': 6, 'honest': 1, '’': 16, 'answer': 2, 'tell': 2, 'right': 4, 'cours': 1, 'action': 3, ';': 1, 'onli': 3, 'know': 5, 'import': 1, 'discourag': 1, 'failur': 4, 'pursu': 3, 'goal': 4, 'dream': 6, 'sinc': 1, 'mean': 4, 'differ': 3, 'thing': 2, 'peopl': 3, 'person': 4, 'fix': 1, 'mindset': 2, 'blow': 1, 'self-esteem': 1, 'yet': 2, 'growth': 1, 'opportun': 1, 'improv': 1, 'find': 2, 'new': 1, 'way': 2, 'overcom': 2, 'obstacl': 1, 'respons': 1, 'wrong': 1, 'neither

## Generate Inverse Document Frequency matrix

What we really need is the Inverse Document Frequency. Which is defined as the log inverse of averaged document frequency. Log (Document frequency when divided by total number of docs in the corpus and inversed), gives us the IDF. 

Here I implement Laplace smoothing by adding 1 to the document frequency of a word. This is to make sure that the denominator doesn't go to zero when we see a new word (during vectorisation of new document) not previously seen while creating the model. 

In [63]:
# calculate IDF 
import math
def calc_inverse_doc_frequency(perdoc_term_freq_matrix, doc_frequency, total_documents):
    
    idf = {}
    for doc, table in perdoc_term_freq_matrix.items():
        idf_={}
        for word, count in table.items():
            idf_[word] = math.log10(total_documents/doc_frequency[word]+1)
        idf[doc] = idf_
    return idf

inverse_doc_freq = calc_inverse_doc_frequency(perdoc_term_frequency_matrix, doc_frequency, total_docs)
print(inverse_doc_freq)
            

{'Those Who Are R': {'resili': 1.4313637641589874, 'stay': 1.4313637641589874, 'game': 1.2632414347745813, 'longer': 1.4313637641589874, '“': 1.0569048513364727, 'mountain': 1.724275869600789, 'truth': 1.724275869600789, 'never': 1.4313637641589874, 'climb': 1.724275869600789, 'vain': 1.724275869600789, ':': 0.8750612633917001, 'either': 1.724275869600789, 'reach': 1.724275869600789, 'point': 1.4313637641589874, 'higher': 1.724275869600789, 'today': 1.724275869600789, ',': 0.52680903890877, 'train': 1.724275869600789, 'power': 1.146128035678238, 'abl': 1.724275869600789, 'tomorrow.': 1.724275869600789, '”': 1.0569048513364727, '—': 1.2632414347745813, 'friedrich': 1.724275869600789, 'nietzsch': 1.724275869600789, 'challeng': 1.4313637641589874, 'setback': 1.4313637641589874, 'meant': 1.724275869600789, 'defeat': 1.2632414347745813, 'promot': 1.724275869600789, '.': 0.33355922049090114}, 'However, I real': {'howev': 1.4313637641589874, ',': 0.52680903890877, 'realis': 1.4313637641589874

## Generate TF-IDF matrix

Multiplying TF of a word and IDF of a word gives us the TF-IDF score. This is implemented below. 

In [64]:
def create_tf_idf_matrix(term_freq, inv_doc_freq):
    
    tf_idf = {}
    
    for (doc1, table1), (doc2, table2) in zip(term_freq.items(), inv_doc_freq.items()):
        tf_idf_ = {}
        for (word1, count1), (word2, count2) in zip(table1.items(), table2.items()):
            tf_idf_[word1] = count1 * count2
        tf_idf[doc1] = tf_idf_
        
    return tf_idf

tf_idf_matrix = create_tf_idf_matrix(term_frequency, inverse_doc_freq)
tf_idf_matrix

{'Those Who Are R': {'resili': 0.04617302465028991,
  'stay': 0.04617302465028991,
  'game': 0.04074972370240584,
  'longer': 0.04617302465028991,
  '“': 0.0340937048818217,
  'mountain': 0.05562180224518674,
  'truth': 0.05562180224518674,
  'never': 0.04617302465028991,
  'climb': 0.05562180224518674,
  'vain': 0.05562180224518674,
  ':': 0.02822778269005484,
  'either': 0.05562180224518674,
  'reach': 0.05562180224518674,
  'point': 0.04617302465028991,
  'higher': 0.05562180224518674,
  'today': 0.05562180224518674,
  ',': 0.01699383996479903,
  'train': 0.05562180224518674,
  'power': 0.03697187211865283,
  'abl': 0.05562180224518674,
  'tomorrow.': 0.05562180224518674,
  '”': 0.0340937048818217,
  '—': 0.04074972370240584,
  'friedrich': 0.05562180224518674,
  'nietzsch': 0.05562180224518674,
  'challeng': 0.04617302465028991,
  'setback': 0.04617302465028991,
  'meant': 0.05562180224518674,
  'defeat': 0.04074972370240584,
  'promot': 0.05562180224518674,
  '.': 0.01075997485454

## Generate a document scorer

Below is a function that uses the TF-IDF matrix to score a document. The score of a document is the average of scores of it's constituent unique words 

In [65]:
def doc_scorer(tf_idf_matrix):
    
    doc_scores = {}
    for doc, table in tf_idf_matrix.items():
        doc_score = 0
        for word, count in table.items():
            doc_score += count
        doc_scores[doc] = doc_score/len(table)
    return doc_scores

doc_scores = doc_scorer(tf_idf_matrix)
doc_scores

{'Those Who Are R': 0.04648155666866914,
 'However, I real': 0.09337584558849191,
 'Have you experi': 0.33044027578858176,
 'To be honest, I': 0.18577587292839035,
 'I can’t tell yo': 0.130510856919329,
 'However, it’s i': 0.07883421102714701,
 'To a person wit': 0.07633345382224603,
 'Same failure, y': 0.1784827045447852,
 'Who is right an': 0.42840896093981334,
 'Neither.': 0.5144587725229225,
 'Each person has': 0.17880195618411163,
 'Those who are r': 0.14712266033063268,
 'I’ve coached ma': 0.10802406605565623,
 'It was at that ': 0.23247539882253454,
 'Perhaps all tho': 0.22455218414251374,
 'It was the 19th': 0.04022160619251501,
 'Consider the ad': 0.0603717169621282,
 'Even more than ': 0.04628580442205354,
 'Some of you rea': 0.13766001132439434,
 'For others, at ': 0.1406875100532238,
 'What I wish to ': 0.14295394223408034,
 'If you settle f': 0.14396608002242173,
 '“Two people on ': 0.030753323118267725,
 'Don’t leave it ': 0.11230937679284027,
 'It must come fr': 0.307535

## Threshold the scores to select the top k docs 

In [80]:
threshold = np.mean(list(doc_scores.values()))
print(threshold)

0.1623437110810388


In [83]:
docs_in_summary = [(doc, score) for doc, score in doc_scores.items() if score > threshold]
print(docs_in_summary)

[('Have you experi', 0.33044027578858176), ('To be honest, I', 0.18577587292839035), ('Same failure, y', 0.1784827045447852), ('Who is right an', 0.42840896093981334), ('Neither.', 0.5144587725229225), ('Each person has', 0.17880195618411163), ('It was at that ', 0.23247539882253454), ('Perhaps all tho', 0.22455218414251374), ('It must come fr', 0.30753516365060407), ('Gnaw away at yo', 0.18821082220097304), ('Where are you s', 0.26532590509565485), ('Could you be yo', 0.3035787246456259), ('Success is a fi', 0.1855056030648756), ('So become inten', 0.22768926965061556), ('Commit to it.', 0.44123074616247215), ('Nurture your dr', 0.30557774753657574), ('Don’t leave you', 0.19738968792383504)]


In [87]:
# Sort the tuple list by scores and extract the full sentences
sorted_docs_in_summary = sorted(docs_in_summary, key=lambda x:x[1], reverse=True)
print(sorted_docs_in_summary)

[('Neither.', 0.5144587725229225), ('Commit to it.', 0.44123074616247215), ('Who is right an', 0.42840896093981334), ('Have you experi', 0.33044027578858176), ('It must come fr', 0.30753516365060407), ('Nurture your dr', 0.30557774753657574), ('Could you be yo', 0.3035787246456259), ('Where are you s', 0.26532590509565485), ('It was at that ', 0.23247539882253454), ('So become inten', 0.22768926965061556), ('Perhaps all tho', 0.22455218414251374), ('Don’t leave you', 0.19738968792383504), ('Gnaw away at yo', 0.18821082220097304), ('To be honest, I', 0.18577587292839035), ('Success is a fi', 0.1855056030648756), ('Each person has', 0.17880195618411163), ('Same failure, y', 0.1784827045447852)]


In [91]:
# Extract and print the summary

summary = []

for doc, score in sorted_docs_in_summary:
    for odoc in docs:
        if odoc.startswith(doc):
            summary.append(odoc)

print(summary)


['Neither.', 'Commit to it.', 'Who is right and who is wrong?', 'Have you experienced this before?', 'It must come from within you.', 'Nurture your dreams.', 'Could you be you playing for bigger stakes than you are?', 'Where are you settling in your life right now?', 'It was at that point their biggest breakthrough came.', 'So become intentional on what you want out of life.', 'Perhaps all those years of perseverance finally paid off.', 'Don’t leave your dreams to chance.', 'Gnaw away at your problems until you solve them or find a solution.', 'To be honest, I don’t have the answers.', 'Success is a fickle and long game with highs and lows.', 'Each person has a different mindset that decides their outcome.', 'Same failure, yet different responses.']
